## 1. Import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from IPython.display import clear_output
!unzip /content/drive/MyDrive/ML_notes/301_Final_proj/images2.zip -d /content/images
clear_output()

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: /content/images/images/62764_2019.jpg  
  inflating: /content/images/images/62765_2019.jpg  
  inflating: /content/images/images/62766_2019.jpg  
  inflating: /content/images/images/62767_2019.jpg  
  inflating: /content/images/images/62768_2019.jpg  
  inflating: /content/images/images/62769_2019.jpg  
  inflating: /content/images/images/6276_2003.jpg  
  inflating: /content/images/images/62770_2019.jpg  
  inflating: /content/images/images/62771_2019.jpg  
  inflating: /content/images/images/62772_2019.jpg  
  inflating: /content/images/images/62773_2019.jpg  
  inflating: /content/images/images/62774_2019.jpg  
  inflating: /content/images/images/62775_2019.jpg  
  inflating: /content/images/images/62776_2019.jpg  
  inflating: /content/images/images/62777_2019.jpg  
  inflating: /content/images/images/62778_2019.jpg  
  inflating: /content/images/images/62779_2019.jpg  
  inflating: /content/images/images/6277_2003.jpg  
  inflating: /content

### 2. Pipeline for data augmentation & Inspect example input images

In [ ]:
#libraries
import matplotlib.pyplot as plt
import torch
from numpy import array
from scipy import misc
import os
import numpy as np
%matplotlib inline
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

In [ ]:
data_dir = '/content/images/'
image_size = 64
batch_size = 128
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")


train_ds = ImageFolder(data_dir, transform=transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(*stats)]))

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=3, pin_memory=True)

In [ ]:
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
device

device(type='cuda', index=0)

In [ ]:
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device       
    def __iter__(self):
        for b in self.dl: 
            yield to_device(b, self.device)
    def __len__(self):
        return len(self.dl)

**Visualize some images from original dataset:**

In [ ]:
def denorm(img_tensors):
    return img_tensors * 0.5 + 0.5

def show_images(images):
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_xticks([]); ax.set_yticks([])
    ax.imshow(make_grid(denorm(images.detach()[:36]), nrow=6).permute(1, 2, 0))

def show_batch(dl):
    for images, _ in dl:
        show_images(images)
        break

In [ ]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [ ]:
print(color.BOLD + color.UNDERLINE + color.DARKCYAN + 'Example input images:' + color.END)

show_batch(train_dl)

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)